# How to use the ifes_apt_tc_data_modeling library

## Load ifes_apt_tc_data_modeling library

In [ ]:
import os
import numpy as np
import h5py
from jupyterlab_h5web import H5Web
from ifes_apt_tc_data_modeling.utils.utils import isotope_to_hash, hash_to_isotope
from ifes_apt_tc_data_modeling.utils.definitions import NEUTRON_NUMBER_FOR_ELEMENT
from ifes_apt_tc_data_modeling.utils.nx_ion import NxIon
from ifes_apt_tc_data_modeling.utils.molecular_ions import MolecularIonBuilder
# utility function for exporting (molecular) ions into a simple HDF5 file.
from ifes_apt_tc_data_modeling.utils.hfive import simple_hfive_file
print(f"{os.getcwd()}")

***

## Imago *.analysis reader

In [ ]:
prefix = f"{os.getcwd()}/data/imago/examples_without_provenance"
fnm = ["default.analysis"]

In [ ]:
from ifes_apt_tc_data_modeling.imago.imago_reader import ReadImagoAnalysisFileFormat

for fpath in fnm:
    if fpath.lower().endswith(".analysis"):
        print(fpath)
        imago = ReadImagoAnalysisFileFormat(f"{prefix}/{fpath}")
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry1")
            idx = 1
            for ion in imago.imago["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

## CAMECA HDF5 reader

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/cameca_hfive/examples_with_provenance"
fnm = ["R69_152985-v01_Top_Level_ROI.hdf5"]

In [ ]:
from ifes_apt_tc_data_modeling.cameca.cameca_reader import ReadCamecaHfiveFileFormat

for fpath in fnm:
    if fpath.lower().endswith(".hdf5"):
        print(fpath)
        cameca = ReadCamecaHfiveFileFormat(f"{prefix}/{fpath}")
        print(cameca.get_mass_to_charge_state_ratio())
        print(cameca.get_reconstructed_positions())
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry1")
            idx = 1
            for ion in cameca.rng["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

## CSV reader

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/csv/examples_without_provenance"
fnm = ["Annealed CoCrNi_100.csv"]

In [ ]:
from ifes_apt_tc_data_modeling.csv.csv_reader import ReadCsvFileFormat

csv = ReadCsvFileFormat(f"{prefix}/{fnm[0]}")
print(csv.file_path)
print(csv.file_size)
xyz = csv.get_reconstructed_positions()
print(np.shape(xyz.magnitude))
mq = csv.get_mass_to_charge_state_ratio()
print(np.shape(mq.magnitude))

## GPM/Rouen ENV system files

<div class="alert alert-block alert-danger">An ENV file has to be formatted using the UTF8 character encoding for this reader to function!</div>

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/env/examples_without_provenance"
fnm = ["ErMnO.env"]

In [ ]:
from ifes_apt_tc_data_modeling.env.env_reader import ReadEnvFileFormat

In [ ]:
env = ReadEnvFileFormat(f"{prefix}/{fnm[0]}")
for m_ion in env.env["molecular_ions"]:
    m_ion.report()

## FAU/Erlangen pyccapt control/calibration/ranging HDF5 files

In [ ]:
# eventually change needed point to the location of the files to analyze
prefix = f"{os.getcwd()}/../tests/data/env/examples_without_provenance"
fnm = ["1748_Nov-14-2023_13-31_Al.h5", "1748_Al.h5", "1748_Al_range_.h5"]
# df = pd.read_hdf(f"{prefix}/{fnm[1]}")
# H5Web(f"{prefix}/{fnm[1]}")

In [ ]:
from ifes_apt_tc_data_modeling.pyccapt.pyccapt_reader import (
    ReadPyccaptControlFileFormat,
    ReadPyccaptCalibrationFileFormat,
    ReadPyccaptRangingFileFormat,
)

In [ ]:
# the actual measurement (comparable to LEAP's STR/RRAW/RHIT/HITS)
pyc_m = ReadPyccaptControlFileFormat(f"{prefix}/{fnm[0]}")

# the calibrations (voltage, bowl, m/q, comparable to LEAP's ROOT and POS/EPOS)
pyc_c = ReadPyccaptCalibrationFileFormat(f"{prefix}/{fnm[1]}")
xyz = pyc_c.get_reconstructed_positions()
print(xyz.magnitude)
m_q = pyc_c.get_mass_to_charge_state_ratio()
print(m_q.magnitude)

# the ranging definitions (comparable to RNG/RRNG)
pyc_r = ReadPyccaptRangingFileFormat(f"{prefix}/{fnm[2]}")

## ATO

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/ato/examples_without_provenance"
compare = False  # True for APSuite converted ATO to EPOS checks
fnm = [
    "Si.epos.v3.ATO",
    "Si.epos.v5.ATO",
]  # Si.epos example converted using tool from Cameca/APSuite from EPOS>ATO v3 and v5
if not compare:
    fnm = [
        "finfet_system.ato",
        "multilayer_system.ato",
        "R31_01853-v11_Fig2.ATO",
        "R31_03988-v05_Fig3.ATO",
        "R31_03992-v02_S2(a).ATO",
        "R31_03995-v02_S2(b).ATO",
        "R31_11378-v01.ato",
        "R31_11381-v02.ato",
        "R31_11553-v01.ato",
        "R31_11554-v01.ato",
        "R31_11556-v01.ato",
    ]
for fpath in fnm:
    print(os.path.getsize(f"{prefix}/{fpath}"))

In [ ]:
from ifes_apt_tc_data_modeling.ato.ato_reader import ReadAtoFileFormat

# the next line only to compare with ePOS
from ifes_apt_tc_data_modeling.epos.epos_reader import ReadEposFileFormat

In [ ]:
for fpath in fnm:
    print(f"{prefix}/{fpath}")
    ato = ReadAtoFileFormat(f"{prefix}/{fpath}")
    xyz = ato.get_reconstructed_positions()
    for idx, dim in enumerate(["x", "y", "z"]):
        print(f"{dim}: {np.min(xyz.magnitude[:, idx])}, {np.max(xyz.magnitude[:, idx])}")
    mq = ato.get_mass_to_charge_state_ratio()
    print(f"m/q: {np.min(mq.magnitude)}, {np.max(mq.magnitude)}")

    # compare with original Si.epos
    if compare:
        epos = ReadEposFileFormat(f"{prefix}/Si.epos")
        print(epos.file_path)
        print(epos.file_size)
        xyz = epos.get_reconstructed_positions()
        for idx, dim in enumerate(["x", "y", "z"]):
            print(f"{dim}: {np.min(xyz.magnitude[:, idx])}, {np.max(xyz.magnitude[:, idx])}")
        mq = epos.get_mass_to_charge_state_ratio()
        print(f"m/q: {np.min(mq.magnitude)}, {np.max(mq.magnitude)}")

## FAU Erlangen Atomprobe Matlab Toolbox FIG

<div class="alert alert-block alert-danger">A Matlab FIG has to be first processed using the matlab_fig_to_txt.m script for this reader to function!</div>

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/fig/examples_without_provenance/ger_erlangen_felfer"
fnm = ["R56_01769.rng.fig.txt"]

In [ ]:
from ifes_apt_tc_data_modeling.fig.fig_reader import ReadFigTxtFileFormat

In [ ]:
for fpath in fnm:
    if fpath.lower().endswith(".fig.txt"):
        print(fpath)
        figtxt = ReadFigTxtFileFormat(f"{prefix}/{fpath}")
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry1")
            idx = 1
            for ion in figtxt.fig["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

Display generated HDF5 file using H5Web.

In [ ]:
H5Web(f"{prefix}/{fpath}.nxs")

## RRNG

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/rrng/examples_without_provenance/ger_duesseldorf_kuehbach"
prefix = f"{os.getcwd()}/../tests/data/rrng/examples_without_provenance/ger_aachen_saelker"
# prefix = f"{os.getcwd()}/data/rrng/examples_without_provenance/ger_berlin_kuehbach"
fnm = ["R31_06365-v02.rrng"]
fnm = ["VAlN_film_plan-view_700C.rrng"]
# fnm = ["Ranges_R45_2434-v01.rrng"]
# fnm = ["TiAlN_film_cross-section_850C.rrng"]

In [ ]:
from ifes_apt_tc_data_modeling.rrng.rrng_reader import ReadRrngFileFormat

In [ ]:
# for dirpath, dirnames, filenames in os.walk(prefix):
#    for fpath in filenames:
for fpath in fnm:
    if fpath.lower().endswith(".rrng"):
        print(fpath)
        rrng = ReadRrngFileFormat(f"{prefix}/{fpath}", unique=True)  # , verbose=True)
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry1")
            idx = 1
            for ion in rrng.rrng["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

In [ ]:
H5Web(f"{prefix}/{fpath}.nxs")

## RNG

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/rng/examples_without_provenance/ger_duesseldorf_kuehbach"
fnm = ["SeHoKim_R5076_44076_v02.rng"]

In [ ]:
from ifes_apt_tc_data_modeling.rng.rng_reader import ReadRngFileFormat

In [ ]:
for fpath in fnm:
    if fpath.lower().endswith(".rng"):
        print(fpath)
        rng = ReadRngFileFormat(f"{prefix}/{fpath}")
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry")
            idx = 1
            for ion in rng.rng["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

In [ ]:
H5Web(f"{prefix}/{fpath}.nxs")

## POS

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/pos/examples_without_provenance"
fnm = ["ErMnO_pole.pos"]

In [ ]:
from ifes_apt_tc_data_modeling.pos.pos_reader import ReadPosFileFormat

In [ ]:
for fpath in fnm:
    pos = ReadPosFileFormat(f"{prefix}/{fpath}")
    print(pos.file_path)
    print(pos.file_size)
    xyz = pos.get_reconstructed_positions()
    print(np.shape(xyz.magnitude))
    mq = pos.get_mass_to_charge_state_ratio()
    print(np.shape(mq.magnitude))

## ePOS

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/epos/examples_without_provenance"
fnm = ["R18_58152-v02.epos"]

In [ ]:
from ifes_apt_tc_data_modeling.epos.epos_reader import ReadEposFileFormat

In [ ]:
for fpath in fnm:
    epos = ReadEposFileFormat(f"{prefix}/{fpath}")
    print(epos.file_path)
    print(epos.file_size)
    xyz = epos.get_reconstructed_positions()
    print(np.shape(xyz.magnitude))
    mq = epos.get_mass_to_charge_state_ratio()
    print(np.shape(mq.magnitude))
    raw_tof = epos.get_raw_time_of_flight()
    print(np.shape(raw_tof.magnitude))
    st_v = epos.get_standing_voltage()
    print(np.shape(st_v.magnitude))
    p_v = epos.get_pulse_voltage()
    print(np.shape(p_v.magnitude))
    det_xy = epos.get_hit_positions()
    print(np.shape(det_xy.magnitude))
    n_p = epos.get_number_of_pulses()
    print(np.shape(n_p.magnitude))
    h_p = epos.get_ions_per_pulse()
    print(np.shape(h_p.magnitude))

## APT

In [ ]:
prefix = f"{os.getcwd()}/../tests/data/apt/examples_without_provenance"
fnm = ["70_50_50.apt"]

In [ ]:
from ifes_apt_tc_data_modeling.apt.apt6_reader import ReadAptFileFormat

In [ ]:
for fpath in fnm:
    apt = ReadAptFileFormat(f"{prefix}/{fpath}")
    print(apt.get_metadata_table())
    xyz = apt.get_reconstructed_positions()
    print(np.shape(xyz.magnitude))
    mq = apt.get_mass_to_charge_state_ratio()
    print(np.shape(mq.magnitude))

## Testing molecular ion builder and charge state analysis

In [ ]:
mion = MolecularIonBuilder(
    min_abundance=0.0,
    min_abundance_product=0.0,
    min_half_life=np.inf,
    sacrifice_uniqueness=True,
    verbose=True,
)
# issue 5 Range33=187.4800 190.2560 Vol:0.12084 Ce:1 O:3 Color:00FF00 is another example with many such duplicates
mion.combinatorics(
    [
        isotope_to_hash(58, NEUTRON_NUMBER_FOR_ELEMENT),
        isotope_to_hash(8, NEUTRON_NUMBER_FOR_ELEMENT),
        isotope_to_hash(8, NEUTRON_NUMBER_FOR_ELEMENT),
        isotope_to_hash(8, NEUTRON_NUMBER_FOR_ELEMENT),
    ],
    187.4800,
    190.2560,
)

***
Markus Kühbach, 2025/12/03<br>
<br>
<a href="https://www.fairmat-nfdi.eu/fairmat">FAIRmat</a> is a consortium on research data management which is part of the German NFDI.<br>
The project is funded by the Deutsche Forschungsgemeinschaft (DFG, German Research Foundation) – project 460197019.